In [1]:
import pandas as pd
import numpy as np

In [2]:
def transform(file):
    r = pd.DataFrame()
    r['index'] = file['Unnamed: 0']
    r['resultados_24'] = file['currentResults']

    d = {
        'territoryKey': file[0].territoryKey,
        'território': file[0].territoryFullName,
        'resultados': r
    }

    row = pd.DataFrame(data=d, index='territoryKey')

    return row

In [3]:
local = pd.read_csv('datasets/territory_codes_local.csv')
foreign = pd.read_csv('datasets/territory_codes_foreign.csv')

codes = (
    pd.concat([local, foreign])
    .rename(columns={'name': 'freguesia'})
)

codes

,territoryKey,freguesia,concelho,distrito
0,LOCAL-430101,Altares,Angra do Heroísmo,Açores
1,LOCAL-430102,Angra (Nossa Senhora da Conceição),Angra do Heroísmo,Açores
2,LOCAL-430103,Angra (Santa Luzia),Angra do Heroísmo,Açores
3,LOCAL-430104,Angra (São Pedro),Angra do Heroísmo,Açores
4,LOCAL-430105,Angra (Sé),Angra do Heroísmo,Açores
...,...,...,...,...
19,FOREIGN-930299,Postos Consulares da China,China,Fora da Europa
20,FOREIGN-920799,Postos Consulares dos Estados Unidos da América,Estados Unidos da América,Fora da Europa
21,FOREIGN-919999,Postos Consulares dos países de África,Países de África,Fora da Europa
22,FOREIGN-929999,Restantes Postos Consulares Dos Países da América,Restantes Países da América,Fora da Europa


In [43]:
t = pd.DataFrame(columns=['territoryKey', 'território', 'resultados'])
t['territoryKey'] = codes['territoryKey']

file = (
    pd.read_csv('datasets\Local\Açores\Angra do Heroísmo\Altares.csv')
    .set_index(0)
)

file


KeyError: 'None of [0] are in the columns'

In [32]:

r = pd.DataFrame()

r = file.loc
#r['resultados_24'] = file.loc[1]

r


In [ ]:

d['resultados']

t.loc[t['territoryKey'] == d['territoryKey'], 'território'] = d['território']
t.loc[t['territoryKey'] == d['territoryKey'], 'resultados'] = d['resultados'].to_dict()

r

In [ ]:
# results = pd.DataFrame(columns=['territoryKey', 'território', 'resultados'])

# for i, row in codes.iterrows():
#     tkey = row['territoryKey']
#     loc = tkey.split('-')[0].capitalize()
#     dist = row['distrito']
#     county = row['concelho']
#     parish = row['freguesia']

#     path = 'datasets/' + loc + '/' + dist + '/' + county + '/' + parish + '.csv'

#     file = pd.read_csv(path)
#     row = transform(file)

#     results.loc[len(results.index)] = row

# results